<a href="https://colab.research.google.com/github/davidislip/RedBlueSetCovering/blob/main/Deterministic_RBSC___No_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stochastic Red Blue Set Covering 

### Deterministic Approximation Algorithms Implementation

* Step 1: a function that generates instances of the red-blue set covering 

* Step 2: Carr et. al. Approximation Algorithm 

* Step 3: Peleg. Approximation Algorithm 


In [7]:
!sudo python -m pip install gurobipy==9.1.2
!pip install netgraph
import gurobipy as gp
import pandas as pd
import numpy as np
from gurobipy import GRB
from google.colab import drive 
from itertools import product
import math, sys, time
from netgraph import Graph, InteractiveGraph, EditableGraph
import matplotlib.pyplot as plt
drive.mount('/content/gdrive')
pth = 'gdrive/My Drive/Colab Notebooks/'
sys.path.append(pth + 'RBSC/')
%matplotlib inline

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#read in licence info
gurobi_licence = pd.read_csv(pth +'RBSC/gurobi.csv')
print("Required info for Gurobi:", gurobi_licence.columns)
try:
  #web license try to access it via uoft
  e = gp.Env(empty=True)
  #e.setParam('OutputFlag', 0)
  e.setParam('WLSACCESSID', gurobi_licence.WLSACCESSID[0])
  e.setParam('LICENSEID', gurobi_licence.LICENSEID[0])
  e.setParam('WLSSECRET', gurobi_licence.WLSSECRET[0])
  e.start()
except: 
  !chmod 755 /content/gdrive/My\ Drive/Colab\ Notebooks/SVM\ MVO/gurobi/grbgetkey
  !/content/gdrive/My\ Drive/Colab\ Notebooks/SVM\ MVO/gurobi/grbgetkey {gurobi_licence.LOCAL[0]}
  e = gp.Env(empty=True)
  #chmod 755 grbgetkey
  e.start()

Required info for Gurobi: Index(['WLSACCESSID', 'LICENSEID', 'WLSSECRET', 'LOCAL'], dtype='object')
Changed value of parameter WLSACCESSID
Changed value of parameter LICENSEID
Changed value of parameter WLSSECRET
info  : grbgetkey version 9.1.2, build v9.1.1rc0-30-g8af970cb
info  : Contacting Gurobi license server...
info  : License file for license ID 726785 was successfully retrieved
info  : License expires at the end of the day on 2022-03-20
info  : Saving license file...

In which directory would you like to store the Gurobi license file?
[hit Enter to store it in /opt/gurobi]: 

info  : License 726785 written to file /opt/gurobi/gurobi.lic

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-03-20
Using license file /opt/gurobi/gurobi.lic


In [8]:
import networkx as nx
import random
N = 15 #number of sets 
rho = 100 #number of reds
beta = 7 #number of blue 
CoverFactor = 1
l = 2 #scaling factor for plots 

#generate N sets such all the blues are contained in at least one set 

Reds = {i:'R' for i in range(rho)}
Blues = {i:'B' for i in range(rho, rho+beta)}
Elements = {**Reds, **Blues}
Sets = {}

In [9]:
k = 0
UncoveredBlues = [b for b in Blues.keys()]
#randomly sample until all the blues are covered 
while (k < N or UncoveredBlues != []):
  n_k = random.sample(range(1,len(Elements)//CoverFactor),1)[0]
  DoesNotCoverAnyBlue = True
  while DoesNotCoverAnyBlue:
    SetCandidate = random.sample(list(Elements.keys()), n_k)
    BlueCoveredBool = [b in SetCandidate for b in Blues.keys()]
    if True in BlueCoveredBool:
      DoesNotCoverAnyBlue = False
  if UncoveredBlues == []:
    ind = 'Set' + str(k)
    Sets[ind] = SetCandidate
    k = k+1

  CoversNewBlue = False
  for element in Blues.keys():
    if element in SetCandidate and element in UncoveredBlues:
      UncoveredBlues.remove(element)
      CoversNewBlue = True
  if CoversNewBlue:
    ind = 'Set' + str(k)
    Sets[ind] = SetCandidate
    k = k+1


In [10]:
#number of sets generated in this instance
len(Sets)

15

In [11]:
#testing 
Test = set()
for Set in Sets.values():
  Test = Test.union(set(Set))
for b in [e for e in Elements.keys() if Elements[e] == 'B']:
  if b not in Test:
    print("Error Blue element missing")

In [12]:
#assign the final set as set that covers all elements
#Sets['SetTotal'] = list(Elements.keys())

### Direct Implementation 

$$
\begin{equation}
\begin{aligned}
\underset{x,y}{\min}& \sum_{r\in R} y_r &\\
\text{s.t} \quad y_r &\geq x_S &\forall (r,S): r \in S \cap R \\
\sum_{S \ni b} x_S &\geq 1 &\forall b \in B\\
x_S \in \{0,1\}&,\  y_r \geq 0
\end{aligned}
\end{equation}
$$



In [72]:
def DeterministicRedBlue(Reds, Blues, Sets):
  '''
  Direct implementation of the deterministic red blue set covering problem
  '''
  m = gp.Model()
  x = m.addVars(Sets.keys(), vtype = GRB.BINARY)
  y = m.addVars(Reds.keys(), vtype = GRB.CONTINUOUS, lb = 0)

  m.setObjective(y.sum(), GRB.MINIMIZE)

  # for r in Reds.keys():
  #   m.addConstrs((y[r] >= x[S] for S in Sets.keys() if r in Sets[S]))

  for S in Sets.keys():
    m.addConstrs((y[r] >= x[S] for r in set(Sets[S]).intersection(set(Reds.keys()))))
  for b in Blues.keys():
    m.addConstr(gp.quicksum(x[S] for S in \
                            {Set for Set in Sets.keys() if b in Sets[Set]})\
                >= 1)
  m.optimize()
  #gety
  SelectedReds = [r for r in y.keys() if y[r].x > 0.5]
  #getx
  SelectedSets = [s for s in x.keys() if x[s].x > 0.5]  

  SolnEdgestoBlue = [(b,S) for S in SelectedSets for b in set(Sets[S]).intersection(set(Blues.keys()))]
  SolnEdgestoRed = [(r,S) for S in SelectedSets for r in set(Sets[S]).intersection(set(Reds.keys()))]
  SolnEdges = SolnEdgestoBlue+SolnEdgestoRed
  return SelectedReds, SelectedSets, SolnEdges

SelectedReds, SelectedSets, SolnEdges = DeterministicRedBlue(Reds, Blues, Sets)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 843 rows, 115 columns and 1738 nonzeros
Model fingerprint: 0xe1ffdcd0
Variable types: 100 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 96.0000000
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 842 rows, 115 columns, 1725 nonzeros
Variable types: 100 continuous, 15 integer (15 binary)

Root relaxation: objective 1.428571e+01, 234 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   14.28571    0   10   96.00000   14.28571  85.1%     -    0s
H    0     0                      73.0000000   14.285

### Convert the problem to one where each set has only one blue element

* In this case the follow constraint is valid for a minimal solution: $y_r \geq \sum_{S \ni {r,b}} x_S \quad \forall (r,b) \in R \times B$ 

* In the case that there are multiple sets covering $b$ then all but one of the sets can be discarded at non excess cost. 

* Another inequality is added to ensure that the approximation algorithm guarantees and $2 \sqrt{n k}$ bound where $ k = \max_{i} |S_i \cap B|$: For $b \in B$, let $\mu_b = \min_{S \ni b} |S \cap R|$ then it follows that: 

$$
\begin{equation}
\sum_{r \in R} y_r \geq \mu_b \quad \forall b \in B
\end{equation}
$$

In [31]:
### problem augmentation

def FormAugmentedProblem(Reds, Blues, Sets):
  '''
  Perform the set transformation in Carr et. al.
  '''
  AugmentedSets = {}
  k = 0
  for S in Sets.keys():
    if len(set(Blues.keys()).intersection(Sets[S])) > 1:
      RedsInS = set(Reds.keys()).intersection(Sets[S])
      for b in set(Blues.keys()).intersection(Sets[S]):
        AugmentedSets['Set'+str(k)] = list(RedsInS) + [b]
        k  =k+1
    else:
        AugmentedSets['Set'+str(k)] = Sets[S]
    k = k+1
  return AugmentedSets
AugmentedSets = FormAugmentedProblem(Reds, Blues, Sets)
AugmentedGraph = nx.Graph()
AugmentedGraph.add_nodes_from(Elements)
AugmentedGraph.add_nodes_from({S:"Set" for S in AugmentedSets.keys()})
AugmentedGraph.add_edges_from([(Elem, S) for S in AugmentedSets.keys() for Elem in AugmentedSets[S]]) #element then set 

In [40]:
def DeterministicRedBlueAugmented(Reds, Blues, AugmentedSets, Relax = False):
  ### Solving the augmented model
  m = gp.Model()
  if Relax:
    x = m.addVars(AugmentedSets.keys(), vtype = GRB.CONTINUOUS, lb = 0, ub = 1)
  else:
    x = m.addVars(AugmentedSets.keys(), vtype = GRB.BINARY)
  y = m.addVars(Reds.keys(), vtype = GRB.CONTINUOUS, lb = 0)

  m.setObjective(y.sum(), GRB.MINIMIZE)

  for b in Blues.keys():
    #reds that are in sets that contain b 
    RedsinSContainsb = [set(AugmentedSets[S]).intersection(set(Reds.keys())) for S in AugmentedSets.keys() if b in AugmentedSets[S]]
    mu_b = min([len(Sb) for Sb in RedsinSContainsb])
    m.addConstr(y.sum() >= mu_b)
    
  for (r, b) in product(Reds.keys(), Blues.keys()):
    #sets containing r and b 
    SetsContainingRandB = {S for S in AugmentedSets.keys() if (r in AugmentedSets[S] and b in AugmentedSets[S])}
    if SetsContainingRandB != set():
      m.addConstr(y[r] >= gp.quicksum(x[S] for S in SetsContainingRandB))


  for S in AugmentedSets.keys():
    m.addConstrs((y[r] >= x[S] for r in set(AugmentedSets[S]).intersection(set(Reds.keys()))))


  for b in Blues.keys():
    m.addConstr(gp.quicksum(x[S] for S in \
                            {Set for Set in AugmentedSets.keys() if b in AugmentedSets[Set]})\
                >= 1)
  m.optimize()
  #gety
  SelectedReds = [r for r in y.keys() if y[r].x > 0.5]
  #getx
  SelectedAugSets = [s for s in x.keys() if x[s].x > 0.5]  

  SolnEdgestoBlue = [(b,S) for S in SelectedAugSets for b in set(AugmentedSets[S]).intersection(set(Blues.keys()))]
  SolnEdgestoRed = [(r,S) for S in SelectedAugSets for r in set(AugmentedSets[S]).intersection(set(Reds.keys()))]
  SolnEdges = SolnEdgestoBlue+SolnEdgestoRed
  vals_y = {r:y[r].x for r in y.keys()}
  vals_x = {S:x[S].x for S in x.keys()}
  return SelectedReds, SelectedAugSets, SolnEdges, vals_y, vals_x
SelectedReds, SelectedAugSets, SolnEdges, vals_y, vals_x = DeterministicRedBlueAugmented(Reds, Blues, AugmentedSets)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 5056 rows, 166 columns and 14492 nonzeros
Model fingerprint: 0xed15b7bf
Variable types: 100 continuous, 66 integer (66 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 100.0000000
Presolve removed 1784 rows and 6 columns
Presolve time: 0.08s
Presolved: 3272 rows, 160 columns, 9895 nonzeros
Variable types: 98 continuous, 62 integer (62 binary)

Root relaxation: objective 6.200000e+01, 187 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   62.00000    0   14  100.00000   62.00000  38.0%     -    0s
H    0     0                      94.0000000   

In [45]:
SelectedReds, SelectedAugSets, SolnEdges, vals_y, vals_x = DeterministicRedBlueAugmented(Reds, Blues, AugmentedSets, Relax = True)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 5056 rows, 166 columns and 14492 nonzeros
Model fingerprint: 0x7f3f6490
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 6 rows and 0 columns
Presolve time: 0.04s
Presolved: 5050 rows, 166 columns, 13892 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.000000e+01   0.000000e+00      0s
     253    6.2000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 253 iterations and 0.10 seconds
Optimal objective  6.200000000e+01


In [47]:
#### Processing the LP solution 
def CarrApproximationAlgorithm(Reds, Blues, AugmentedSets, vals_y):
  GoodBlues = set()
  BadBlues = set()
  for b in Blues.keys():
    SetsContainingb = [S for S in AugmentedSets.keys() if b in AugmentedSets[S]]
    if len(SetsContainingb) > n**(0.5) + 0.0001:
      BadBlues.add(b)
    else:
      GoodBlues.add(b)
  yAug = {}
  for r in y.keys():
    yAug[r] = math.floor(vals_y[r]*(n**0.5))
    if yAug[r] > 0.5:
      yAug[r] = 1
  SelectedRedsPhase1 = {r for r in yAug.keys() if yAug[r] > 0.5}
  SelectedAugSetsPhase1  = set()
  for S in AugmentedSets.keys():
    #if the reds in a set are a subset of the phase1 reds
    RedsinS = set(AugmentedSets[S]).intersection(set(Reds.keys()))
    if RedsinS.issubset(SelectedRedsPhase1):
      SelectedAugSetsPhase1.add(S)

  SelectedRedsPhase2 = set() 
  SelectedAugSetsPhase2  = set()
  for b in BadBlues:
    best = None
    best_size = rho
    for S in [Set for Set in AugmentedSets.keys() if (b in AugmentedSets[S])]:
        RedsinS = set(AugmentedSets[S]).intersection(set(Reds.keys()))
        if len(RedsinS) < best_size:
          best = S
          best_size = len(RedsinS)

    if best != None:
        SelectedAugSetsPhase2.add(S)

  for S in SelectedAugSetsPhase2:
    RedsinS = set(AugmentedSets[S]).intersection(set(Reds.keys()))
    SelectedRedsPhase2 = SelectedRedsPhase2.union(RedsinS)

  SelectedAugSets = [s for s in SelectedAugSetsPhase2.union(SelectedAugSetsPhase1)]

  ###Remove redundant sets
  # for b in Blues.keys():
  #   for S1 in [S for S in SelectedAugSets if b in AugmentedSets[S]]:
  #     RedsinS1 = set(AugmentedSets[S1]).intersection(set(Reds.keys()))
  #     RedsinS2 = set()
  #     for S2 in [S for S in SelectedAugSets if (S != S1 and b in AugmentedSets[S])]:
  #       RedsinS2 = RedsinS2.union(set(AugmentedSets[S2]).intersection(set(Reds.keys())))

  #     # print("b ", b)
  #     # print(RedsinS1)
  #     # print(RedsinS2)
  #     if RedsinS1.issubset(RedsinS2):
  #         print("S1 Can be removed")

  #gety
  SelectedReds = [r for r in SelectedRedsPhase2.union(SelectedRedsPhase1)]
  #getx

  SolnEdgestoBlue = [(b,S) for S in SelectedAugSets for b in set(AugmentedSets[S]).intersection(set(Blues.keys()))]
  SolnEdgestoRed = [(r,S) for S in SelectedAugSets for r in set(AugmentedSets[S]).intersection(set(Reds.keys()))]
  SolnEdges = SolnEdgestoBlue+SolnEdgestoRed

  print("Number of reds covered: ", len(SelectedReds))
  return SelectedReds, SelectedAugSets, SolnEdges
  
SelectedReds, SelectedAugSets, SolnEdges = CarrApproximationAlgorithm(Reds, Blues, AugmentedSets, vals_y)

Number of reds covered:  90


### Standard Benders decomposition on the augmented version of the problem (deterministic case)

In [60]:
### Primal subproblem implementation
m_sub = gp.Model()
y = m_sub.addVars(Reds.keys(), obj = 1, name='y', lb = 0, vtype = GRB.CONTINUOUS)

for S in AugmentedSets.keys():
  m_sub.addConstrs((y[r] >= vals_x[S] for r in set(AugmentedSets[S]).intersection(set(Reds.keys()))))

m_sub.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 4342 rows, 100 columns and 4342 nonzeros
Model fingerprint: 0x7b0450fd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 5e-01]
Presolve removed 4342 rows and 100 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds
Optimal objective  4.500000000e+01


In [61]:
### dual subproblem implementation 
m_sub = gp.Model()
#red element set combinatations where r is in S 

RedSetCombos = {(r,S) for S in AugmentedSets.keys() for r in Reds.keys() if r in AugmentedSets[S]}
pi = m_sub.addVars(RedSetCombos, name='pi', lb = 0, vtype = GRB.CONTINUOUS)

for r in Reds.keys():
  SetsContainingr = {Set for Set in AugmentedSets.keys() if r in AugmentedSets[Set]}
  if SetsContainingr!= None:
    m_sub.addConstr(gp.quicksum(pi[(r,S)] for S in SetsContainingr) <= 1)

m_sub.setObjective(gp.quicksum(vals_x[S]*pi[(r,S)] for (r,S) in pi.keys()), GRB.MAXIMIZE)
m_sub.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 100 rows, 4342 columns and 4342 nonzeros
Model fingerprint: 0x0fec14da
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 100 rows and 4342 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.04 seconds
Optimal objective  4.500000000e+01


In [62]:
#!/usr/bin/env python3.7
counter = 0
PrintInfo = False
xWarm = {S:0 for S in vals_x.keys()}
for S in SelectedAugSets:
  xWarm[S] = 1
# Callback - use lazy constraints (and user cuts) to add benders cuts
def benders(model, where):
    if where == GRB.Callback.MIPSOL:    # This is the LAZY CONSTRAINT, called only when an INTEGER solution is found
                                        # (mandatory, you can't find the optimal solution w/o the lazy constraints)
        global counter
        global PrintInfo
        if PrintInfo:
          print('**************************************************')
          print('Integer solution is found...')
       
        vals_x = model.cbGetSolution(x)
        theta = [var for var in model.getVars() if 'theta' in var.varName]
        vals_theta = model.cbGetSolution(theta)
        
        # build the subproblem (it's better to build it once, outside the callback, and then modify it inside - left as an exercise)
        m_sub.setObjective(gp.quicksum(vals_x[S]*pi[(r,S)] for (r,S) in pi.keys()), GRB.MAXIMIZE)

        m_sub.optimize()
        vals_pi = m_sub.getAttr('x', pi)

        if PrintInfo:
          print("\n\n\n")
          print("At iteration " + str(counter) + ", ")
        
          #print("x = ", list(vals_x))
          print("Theta val = ", vals_theta[0])
          print("Master objective value (LB) = ", vals_theta[0])
          #print("pi = ", list(vals_pi.values()))
          print("Benders cut bound = ", sum(vals_x[S]*vals_pi[(r,S)] for (r,S) in vals_pi.keys()))
        
        status = sum(vals_x[S]*vals_pi[(r,S)] for (r,S) in vals_pi.keys()) - vals_theta[0] >= 0.001
        if PrintInfo:
          print("Is their any violation?", status)
        if status:
            if PrintInfo: 
              print('Benders cut added...')
            model.cbLazy(theta[0] >= sum(x[S]*vals_pi[(r,S)] for (r,S) in vals_pi.keys()))
        else:
            print('No violation')
        UB = sum(vals_x[S]*vals_pi[(r,S)] for (r,S) in vals_pi.keys())
        LB = vals_theta[0]
        if PrintInfo:
          print("UB - LB = ", UB - LB)
          print("\n\n\n")

          print('**************************************************')
        
        counter += 1

#subproblem
m_sub = gp.Model()
#red element set combinatations where r is in S 
m_sub.Params.OutputFlag = 0

RedSetCombos = {(r,S) for S in AugmentedSets.keys() for r in Reds.keys() if r in AugmentedSets[S]}
pi = m_sub.addVars(RedSetCombos, name='pi', lb = 0, vtype = GRB.CONTINUOUS)

for r in Reds.keys():
  SetsContainingr = {Set for Set in AugmentedSets.keys() if r in AugmentedSets[Set]}
  if SetsContainingr!= None:
    m_sub.addConstr(gp.quicksum(pi[(r,S)] for S in SetsContainingr) <= 1)


m = gp.Model()

# Create variables
x = m.addVars(AugmentedSets.keys(), vtype = GRB.BINARY, lb = 0, ub = 1, name = 'x')
theta = m.addVar(obj = 1, name='theta')

for b in Blues.keys():
  m.addConstr(gp.quicksum(x[S] for S in \
                          {Set for Set in AugmentedSets.keys() if b in AugmentedSets[Set]})\
              >= 1)
  
#add warm start cuts
# build the subproblem (it's better to build it once, outside the callback, and then modify it inside - left as an exercise)
m_sub.setObjective(gp.quicksum(xWarm[S]*pi[(r,S)] for (r,S) in pi.keys()), GRB.MAXIMIZE)

m_sub.optimize()
vals_pi = {(r,S):pi[(r,S)].x for (r,S) in pi.keys()}
m.addConstr(theta >= sum(x[S]*vals_pi[(r,S)] for (r,S) in vals_pi.keys()))
# m.write('bender-FL.lp')

# Optimize model
m._x = x

m.Params.lazyConstraints = 1 # # To be used with lazy constraints
m.optimize(benders)

vals = m.getAttr('x', x)

print('')
print('Optimal x: %s' % str(vals))
print('Optimal cost: %g' % m.objVal)
print('')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 8 rows, 67 columns and 85 nonzeros
Model fingerprint: 0x7e08d293
Variable types: 1 continuous, 66 integer (66 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 8 rows, 67 columns, 85 nonzeros
Variable types: 1 continuous, 66 integer (66 binary)

Root relaxation: objective 0.000000e+00, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.87520    0    8          -    3.87520      -     -    0s
No violation
H    0     0                     100.0000000    3.8752

A naive benders implementation does not achieve good results, it takes too long. 

### Dual benders cuts formulation on the augmented problem

For a given solution of the master problem $x^*$. In Benders Dual Decomposition cuts are derived by solving:

$$\underset{y,z}{\min} {\Big \{} \sum_{r \in R} y_r \ |\  \sum_{S \ni b} z_S \geq 1\ \forall b,\ y_r \geq z_S \ \forall (S,r): r \in S, z = x^*, y_r \in \mathbb{R}_+, \ z \in \mathbb{R}_+ {\Big \}}$$ (1)

resulting in the following optimality cut:

$$ \theta \geq \sum_{r \in R} \bar{y}_r + \sum_{S} \bar{\lambda}_S (x_S - \bar{z}_S)$$ (2)

where $\bar{\lambda}_S$ is the dual multipler associated with the constraint $x_S = y_S$ for set $S$

Rahmaniani et. al. show that pricing the constrains $x = z$ into the objective function and solving the lagarangian dual problem yields stronger cuts. Let $Y$ denote $\{(y,z) \in \mathbb{R}^{|R|\times|\mathcal{S}|} |  \sum_{S \ni b} z_S \geq 1\ \forall b,\ y_r \geq z_S \ \forall (S,r): r \in S \}$. The lagrangian dual problem takes the following form: 

$$\underset{\lambda}{\max} \underset{y,z \in X}{\min} {\Big \{} \sum_{r \in R} y_r + \sum_{S} \lambda_S (x^*_S - z_S)\  |\  z_S \in \{0,1\} {\Big \}}$$(3)

resullting in the following optimality cut:

$$\theta \geq \sum_{r \in R} \bar{y}_r + \sum_{S} \bar{\lambda}_S (x_S - \bar{z}_S)$$ (4)

The lagarangian problem is challenging to solve. To combat this Rahmaniani et. al. use a aphased approach. The phased approach used by Rahmaniani et. al. begins by generating the first set of optimality cuts by solving (1) and generating (2). Then using the dual values obtained from (1) denoted by $\bar{\lambda}^{(0)}$, the inner minimization in (3) is solved to obtain $\bar{y}^{(0)}, \bar{z}^{(0)}$ and the resulting cut (4) is added. The third phase focuses on heuristically solving the lagrangian dual by lifting the cuts as much as possible. The procedure is as follows: assume that a given series of values (corresponding to cuts) $\bar{y}^{(k)}, \bar{z}^{(k)}, \bar{\lambda}^{(k)}$ $k = 1,2,...,t-1$ has been obtained. The next approximate lagrangian cut is obtained by solving:

$$
\begin{aligned}
\underset{\eta \in \mathbb{R}, \lambda \in \mathbb{R}^{|\mathcal{S}|}}{\max} \quad &\eta - \frac{\delta}{2}||\lambda^{(t-1)} - \lambda||_2^2\\
\text{s.t}\quad &\eta \leq \sum_{r \in R} \bar{y}_r^{(k)} + \sum_{S} \lambda_S(x^*_S - \bar{z}^{(k)}) \quad  k = 1,2,...,t-1
\end{aligned}
$$
(5)

to obtain $\bar{\lambda}^{(t)}$. Then, $\bar{\lambda}^{(t)}$ is used to obtain  $\bar{y}^{(t)}, \bar{z}^{(t)}$  by solving the inner minimization of the  lagrangian dual:

$$\bar{y}^{(t)}, \bar{z}^{(t)} =  \underset{y,z \in X}{\text{argmin}} {\Big \{} \sum_{r \in R} y_r + \sum_{S} \bar{\lambda}^{(t)}_S (x^*_S - z_S)\  |\  z_S \in \{0,1\} {\Big \}}$$(6)


In [64]:
#subproblem implementation for testing 
m_sub = gp.Model()
#red element set combinatations where r is in S 
m_sub.Params.OutputFlag = 1
y = m_sub.addVars(Reds.keys(), obj = 1, name='y', lb = 0, vtype = GRB.CONTINUOUS)
z = m_sub.addVars(AugmentedSets.keys(), obj = 0, name='z', lb = 0, vtype = GRB.CONTINUOUS)

for S in AugmentedSets.keys():
  m_sub.addConstrs((y[r] >= z[S] for r in set(AugmentedSets[S]).intersection(set(Reds.keys()))))

for b in Blues.keys():
  m_sub.addConstr(gp.quicksum(z[S] for S in \
                          {Set for Set in AugmentedSets.keys() if b in AugmentedSets[Set]})\
              >= 1)

LinkConstraint = m_sub.addConstrs((z[S] == 1 for S in z.keys()))### does not matter 
m_sub._LinkConstraint = LinkConstraint #for the call back

m_sub.optimize()

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 4415 rows, 166 columns and 8816 nonzeros
Model fingerprint: 0xa17c773e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4415 rows and 166 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds
Optimal objective  1.000000000e+02


#### Implementation

In [68]:
math.floor(2.1)
math.ceil(2.3)

3

In [73]:
### Phase 1, solve LP relaxation with standard cuts 

#!/usr/bin/env python3.7
counter = 0
PrintInfo = False
IntegerX = True
delta = 1
epsilon  = 0.01 
IterLim = 5
xWarm = {S:0 for S in x.keys()}
for S in SelectedAugSets:
  xWarm[S] = 1

# Callback - use lazy constraints (and user cuts) to add benders cuts
def BDD(model, where):
  global counter
  global PrintInfo
  global IntegerX
  global delta
  global epsilon
  if where == GRB.Callback.MIPSOL:    # This is the LAZY CONSTRAINT, called only when an INTEGER solution is found
                                        # (mandatory, you can't find the optimal solution w/o the lazy constraints)
        if PrintInfo:
          print('**************************************************')
          print('Integer solution is found...')
       
        vals_x = model.cbGetSolution(x)
        theta = [var for var in model.getVars() if 'theta' in var.varName]
        vals_theta = model.cbGetSolution(theta)
        
        # build the subproblem (it's better to build it once, outside the callback, and then modify it inside - left as an exercise)
        for S in vals_x.keys():
          m_sub._LinkConstraint[S].RHS = vals_x[S]

        m_sub.optimize()
        vals_y = m_sub.getAttr('x', y)
        vals_z = m_sub.getAttr('x', z)
        Pi = m_sub.getAttr('Pi', m_sub._LinkConstraint)
        
        if PrintInfo:
          print("\n\n\n")
          print("At iteration " + str(counter) + ", ")
        
          #print("x = ", list(vals_x))
          print("Theta val = ", vals_theta[0])
          print("Master objective value (LB) = ", vals_theta[0])
          #print("pi = ", list(vals_pi.values()))
          print("Benders cut bound = ", m_sub.ObjVal)
        
        status = m_sub.ObjVal - vals_theta[0] >= 0.001
        if PrintInfo:
          print("Is their any violation?", status)
        if status:
            if PrintInfo: 
              print('Benders cut added...')
              print([vals_z[S]*Pi[S] for S in x.keys()])
              print(vals_z)
              print(vals_x)
              print(Pi)
            model.cbLazy(theta[0] >= m_sub.ObjVal 
                         + gp.quicksum(x[S]*Pi[S] for S in x.keys()) 
                         - sum(vals_z[S]*Pi[S] for S in x.keys()))
        else:
            print('No violation')

        UB = m_sub.ObjVal 
        LB = vals_theta[0]
        if PrintInfo:
          print("UB - LB = ", UB - LB)
          print("\n\n\n")

        #if we are at the root node and have added a cut
        if model.cbGet(GRB.Callback.MIPSOL_NODCNT) <= 1 and IntegerX and status:
          #begin phase #2 by solving the inner lagragian

          #first change the variable types for the fractional master problem solutions
          #force them to be integer, let all the already integer variables 
          #be relaxed and free
          J = {S for S in vals_x.keys() if vals_x[S] 
               <= math.ceil(vals_x[S]) - epsilon and
               vals_x[S] >= math.floor(vals_x[S]) + epsilon}
          # for S in J:
          #   z_lagrangian[S].vtype = GRB.CONTINUOUS
          #   z_lagrangian[S].lb = 0
          #   z_lagrangian[S].ub = 1
          #phase 2 solve 2.6 to obtain a cut, then go to phase 3
          m_lagrangian.setObjective(y_lagrangian.sum() 
          - gp.quicksum(z_lagrangian[S]*Pi[S] for S in x.keys()) 
          + gp.quicksum(vals_x[S]*Pi[S] for S in x.keys()) )

          m_lagrangian.optimize()
          vals_y_lagrang = m_lagrangian.getAttr('x', y_lagrangian)
          vals_z_lagrang = m_lagrangian.getAttr('x', z_lagrangian)
          if PrintInfo:
            print('Lagrangian cut added...')
          #add the resulting cut
          model.cbLazy(theta[0] >= m_lagrangian.ObjVal 
                         + gp.quicksum(x[S]*Pi[S] for S in x.keys()) 
                         - sum(vals_z_lagrang[S]*Pi[S] for S in x.keys()))
          
          #project the cut upwards using the heuristic program
          #initialize parameters for loop
          t = 0 
          change = 1
          lambd_last = Pi
          y_last = vals_y_lagrang
          z_last = vals_z_lagrang
          old_eta = m_lagrangian.ObjVal
          delta_t = delta/(t+1)
          #white eta is changing, update the lifting program and then 
          #solve it, then add the resulting strengthed cut to the master
          while t <= IterLim and change > 0.5:
            if PrintInfo:
              print("Lifting Iteration: ", t)
            m_lifting.setObjective(eta  - 
                                   (delta_t/2)*gp.quicksum(
                                       (lambd_last[S] - lambd[S])*(lambd_last[S] - lambd[S])
                                       for S in lambd.keys() ), GRB.MAXIMIZE)
            m_lifting.addConstr(
                eta <= sum(y_last[r] for r in y_last.keys()) + 
                gp.quicksum((vals_x[S] - z_last[S])*lambd[S] for S in lambd.keys())
                )
            
            m_lifting.optimize()
            delta_t = delta/(t+1)
            lambd_last = m_lifting.getAttr('x', lambd)
            new_eta = eta.x
            change = abs(new_eta - old_eta)
            old_eta = new_eta
            m_lagrangian.setObjective(y_lagrangian.sum() 
            - gp.quicksum(z_lagrangian[S]*lambd_last[S] for S in x.keys()) 
            + gp.quicksum(vals_x[S]*lambd_last[S] for S in x.keys()) )

            m_lagrangian.optimize()
            vals_y_lagrang = m_sub.getAttr('x', y)
            vals_z_lagrang = m_sub.getAttr('x', z)
            if PrintInfo:
              print('Lifted Lagrangian cut added...')
              print('eta ', new_eta)
              print('change in eta ', change)
              print('**************************************************')
            model.cbLazy(theta[0] >= m_lagrangian.ObjVal 
                         + gp.quicksum(x[S]*lambd_last[S] for S in x.keys()) 
                         - sum(vals_z_lagrang[S]*lambd_last[S] for S in x.keys()))

            y_last = vals_y_lagrang
            z_last = vals_z_lagrang
            t = t+1

          m_lifting.remove(m_lifting.getConstrs())
          for S in J:#reset lagarangian subproblem 
            z_lagrangian[S].vtype = GRB.BINARY
        counter += 1
  #only at the root node
  if where == GRB.Callback.MIPNODE and model.cbGet(GRB.Callback.MIPNODE_NODCNT) <= 1:    # This is the LAZY CONSTRAINT, called only when an INTEGER solution is found
                                        # (mandatory, you can't find the optimal solution w/o the lazy constraints)
        if PrintInfo:
          print('**************************************************')
          print('Node Relaxation...')
       
        vals_x = model.cbGetNodeRel(x)
        theta = [var for var in model.getVars() if 'theta' in var.varName]
        vals_theta = model.cbGetNodeRel(theta)
        
        # build the subproblem (it's better to build it once, outside the callback, and then modify it inside - left as an exercise)
        for S in vals_x.keys():
          m_sub._LinkConstraint[S].RHS = vals_x[S]

        m_sub.optimize()
        vals_y = m_sub.getAttr('x', y)
        vals_z = m_sub.getAttr('x', z)
        Pi = m_sub.getAttr('Pi', m_sub._LinkConstraint)
        
        if PrintInfo:
          print("\n\n\n")
          print("At iteration " + str(counter) + ", ")
        
          #print("x = ", list(vals_x))
          print("Theta val = ", vals_theta[0])
          print("Master objective value (LB) = ", vals_theta[0])
          #print("pi = ", list(vals_pi.values()))
          print("Benders cut bound = ", m_sub.ObjVal)
        
        status = m_sub.ObjVal - vals_theta[0] >= 0.001
        if PrintInfo:
          print("Is their any violation?", status)
        if status:
            if PrintInfo: 
              print('Benders cut added...')
            model.cbLazy(theta[0] >= m_sub.ObjVal 
                         + gp.quicksum(x[S]*Pi[S] for S in x.keys()) 
                         - gp.quicksum(vals_z[S]*Pi[S] for S in x.keys()))
        else:
            print('No violation')
        UB = m_sub.ObjVal 
        LB = vals_theta[0]
        if PrintInfo:
          print("UB - LB = ", UB - LB)
          print("\n\n\n")

          print('**************************************************')
        
        counter += 1

#subproblem for phase 1
m_sub = gp.Model()
#red element set combinatations where r is in S 
m_sub.Params.OutputFlag = 0
y = m_sub.addVars(Reds.keys(), obj = 1, name='y', lb = 0, vtype = GRB.CONTINUOUS)
z = m_sub.addVars(AugmentedSets.keys(), obj = 0, name='z', lb = 0, vtype = GRB.CONTINUOUS)

for S in AugmentedSets.keys():
  m_sub.addConstrs((y[r] >= z[S] for r in set(AugmentedSets[S]).intersection(set(Reds.keys()))))

for b in Blues.keys():
  m_sub.addConstr(gp.quicksum(z[S] for S in \
                          {Set for Set in AugmentedSets.keys() if b in AugmentedSets[Set]})\
              >= 1)

LinkConstraint = m_sub.addConstrs((z[S] == 1 for S in z.keys()))### does not matter 
m_sub._LinkConstraint = LinkConstraint #for the call back

#subproblem for phase 2 

m_lagrangian = gp.Model()
#red element set combinatations where r is in S 
m_lagrangian.Params.OutputFlag = 0
y_lagrangian = m_lagrangian.addVars(Reds.keys(), obj = 1, name='y', lb = 0, vtype = GRB.CONTINUOUS)
z_lagrangian = m_lagrangian.addVars(AugmentedSets.keys(), obj = 0, name='z', vtype = GRB.BINARY)

for S in AugmentedSets.keys():
  m_lagrangian.addConstrs((y_lagrangian[r] >= z_lagrangian[S] for r in set(AugmentedSets[S]).intersection(set(Reds.keys()))))

for b in Blues.keys():
  m_lagrangian.addConstr(gp.quicksum(z_lagrangian[S] for S in \
                          {Set for Set in AugmentedSets.keys() if b in AugmentedSets[Set]})\
              >= 1)

#heuristic lifting sub problem
m_lifting = gp.Model()

#red element set combinatations where r is in S 
m_lifting.Params.OutputFlag = 0
eta = m_lifting.addVar(name='eta', lb = 0, vtype = GRB.CONTINUOUS)
lambd = m_lifting.addVars(AugmentedSets.keys(), name='lambda', vtype = GRB.CONTINUOUS)


#Master 
m = gp.Model()

# Create variables
x = m.addVars(AugmentedSets.keys(), vtype = GRB.BINARY, name = 'x')
theta = m.addVar(obj = 1, name='theta')
dummy = m.addVar(obj = 1, vtype = GRB.BINARY, name = 'dummy')

for b in Blues.keys():
  m.addConstr(gp.quicksum(x[S] for S in \
                          {Set for Set in AugmentedSets.keys() if b in AugmentedSets[Set]})\
              >= 1)
  
# Optimize model
m._x = x

m.Params.lazyConstraints = 1 # # To be used with lazy constraints
m.optimize(BDD)

vals = m.getAttr('x', x)

print('')
print('Optimal x: %s' % str(vals))
print('Optimal cost: %g' % m.objVal)
print('')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 7 rows, 68 columns and 66 nonzeros
Model fingerprint: 0x293f886a
Variable types: 1 continuous, 67 integer (67 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 7 rows, 68 columns, 66 nonzeros
Variable types: 1 continuous, 67 integer (67 binary)

Root relaxation: objective 0.000000e+00, 7 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.00000    0    1          -    1.00000      -     -    1s
No violation
H    0     0                     100.0000000    1.0000